**SHAKTI SINGH (2311MC13)**
**NLP HMM FROM SCRATCH**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from google.colab import files

In [2]:
file_path = '/content/Brown_train.txt'

In [3]:
def parse_dataset(file_path):
  with open(file_path, 'r') as file:
    sentences = file.read().split('\n\n')
  dataset = [[tuple(word_tag.rsplit('/',1)) for word_tag in sentence.split()] for sentence in sentences if sentence]

  x = []
  y = []
  for i in dataset[0]:
    x.append(i[0])
    y.append(i[1])

  return x,y

In [4]:
df = pd.DataFrame(parse_dataset(file_path))
df = df.T

df.rename(columns = {0:'word',1:'tag'}, inplace = True)
df
start = int(len(df)*0.8)
X_train = df.iloc[:start,0]
X_test = df.iloc[start:,0]
Y_train = df.iloc[:start,1]
Y_test = df.iloc[start:,1]


In [5]:
distinct_tags = set()
i = 0
for i in Y_train:
  for j in (i.split(",")):
    distinct_tags.add(j)

pos_tags = {}
j = 0
for i in distinct_tags:
  pos_tags[i] = j
  j = j + 1

len(pos_tags)


12

In [6]:
# creating transition matrix

count_of_distinct_pos_tags = len(pos_tags)
#declearing transition matrix
transition_count = []
for i in range(0,count_of_distinct_pos_tags):
  t = []
  for j in range (0,count_of_distinct_pos_tags):
    t.append(float(0))
  transition_count.append(t)

count_of_each_pos_tags = [0]*count_of_distinct_pos_tags
pos_tags_as_initial_count = [0]*count_of_distinct_pos_tags
count = 0
for i in Y_train:
  line = list(i.split(","))
  pos_tags_as_initial_count[pos_tags[line[0]]]+=1
  count_of_each_pos_tags[pos_tags[line[0]]]+=1
  length = int(len(line))
  for j in range(1,length):
    count_of_each_pos_tags[pos_tags[line[0]]]+=1
    transition_count[pos_tags[line[j-1]]][pos_tags[line[j]]] += 1


In [7]:
# calculating initial state probability

initial_state_probability = [0]*count_of_distinct_pos_tags
for i in range(count_of_distinct_pos_tags):
  initial_state_probability[i] = (pos_tags_as_initial_count[i] + 1)/count_of_each_pos_tags[i]

# calculate transition probability

transition_probability = copy.deepcopy(transition_count)

for i in range(0,count_of_distinct_pos_tags):
  for j in range(0,count_of_distinct_pos_tags):
    transition_probability[i][j] = ((transition_probability[i][j] + 1)*(100))/(count_of_each_pos_tags[j] + 1)

print(transition_probability)

[[0.0019479507558048932, 0.008522969402539845, 0.004640586570142466, 0.005277880403230063, 0.22371364653243847, 0.0018263839424323782, 0.0009970686182623088, 0.016479894528675015, 0.007134703196347032, 0.0014182586620147782, 0.0018310323360310543, 0.0033041467041136628], [0.0019479507558048932, 0.008522969402539845, 0.004640586570142466, 0.005277880403230063, 0.22371364653243847, 0.0018263839424323782, 0.0009970686182623088, 0.016479894528675015, 0.007134703196347032, 0.0014182586620147782, 0.0018310323360310543, 0.0033041467041136628], [0.0019479507558048932, 0.008522969402539845, 0.004640586570142466, 0.005277880403230063, 0.22371364653243847, 0.0018263839424323782, 0.0009970686182623088, 0.016479894528675015, 0.007134703196347032, 0.0014182586620147782, 0.0018310323360310543, 0.0033041467041136628], [0.0019479507558048932, 0.008522969402539845, 0.004640586570142466, 0.005277880403230063, 0.22371364653243847, 0.0018263839424323782, 0.0009970686182623088, 0.016479894528675015, 0.00713

In [8]:
# Extracting distinct words from the dataset

distinct_words = set()
i = 0
for i in X_train:
  for j in (i.split(" ")):
    distinct_words.add(j)

words = {}
j = 0
for i in distinct_words:
  words[i] = j
  j = j + 1
print(words)
print(len(words))


{'predispositions': 0, 'glycol': 1, 'Hunt': 2, 'stimulation': 3, 'Clean': 4, 'bellows': 5, 'housekeeping': 6, 'reorganized': 7, 'Salvation': 8, 'cooperation': 9, '1819': 10, 'Cascaded': 11, 'lobe': 12, 'Berlitz': 13, 'Larry': 14, 'drown': 15, 'delegation': 16, 'guardian': 17, 'salvage': 18, 'ablated': 19, 'she': 20, 'Rapport': 21, 'hours': 22, 'curved': 23, 'Bee': 24, 'Shall': 25, 'innovate': 26, 'winder': 27, 'insurrection': 28, 'Jacksons': 29, 'first-class': 30, 'replacement': 31, 'cabinet': 32, '1908': 33, 'odds': 34, 'August': 35, 'eye-undeceiving': 36, 'bath': 37, '43*0': 38, 'Gonna': 39, 'Pool': 40, "An'": 41, 'hoping': 42, '4%': 43, 'hoop': 44, 'prose': 45, 'inactive': 46, 'tenants': 47, 'pounds': 48, 'Deor': 49, 'pigments': 50, 'Months': 51, 'printed': 52, 'mainly': 53, 'Spanish-American': 54, 'how': 55, 'delicacy': 56, 'sampled': 57, 'shopkeepers': 58, 'wares': 59, 'shipyards': 60, 'Indeed': 61, 'Blowing': 62, 'anthropological': 63, 'Stirring': 64, 'informing': 65, 'stakes': 6

In [9]:
# creating emission matrix

count_of_distinct_words = len(words)

# declaring emission count matrix
emission_count = []
for i in range(0,count_of_distinct_words):
  t = []
  for j in range(0,count_of_distinct_pos_tags):
    t.append(float(0))
  emission_count.append(t)

for i in range (0,len(X_train)):
  lineX = X_train[i].split(" ")
  lineY = Y_train[i].split(",")
  length = len(lineX)

  for j in range(0, length):
    emission_count[words[lineX[j]]][pos_tags[lineY[j]]] += 1


In [10]:
# calculating emission probability

emission_probability = copy.deepcopy(emission_count)

for i in range(0,count_of_distinct_pos_tags):
  for j in range(0,count_of_distinct_pos_tags):
    emission_probability[i][j] = ((emission_probability[i][j] + 1)*(100))/(count_of_each_pos_tags[j])

print(emission_probability)

# find most frequent pos tag

for i in range(0,count_of_distinct_pos_tags):
  if(count_of_each_pos_tags[i] == max(count_of_each_pos_tags)):
    most_frequent_tag_index = i
    break

print(most_frequent_tag_index)

for i in pos_tags:
  if(pos_tags[i] == most_frequent_tag_index):
    most_frequent_tag = i

print(most_frequent_tag)



[[0.0019479887016655303, 0.008523695874531197, 0.004640801930573603, 0.0052781589781484214, 0.2242152466367713, 0.001826417299824664, 0.001994157119639456, 0.016482610845557938, 0.007135212272565109, 0.001418278776876383, 0.001831065863439108, 0.003304255881575469], [0.0019479887016655303, 0.008523695874531197, 0.004640801930573603, 0.0052781589781484214, 0.2242152466367713, 0.001826417299824664, 0.0029912356794591847, 0.016482610845557938, 0.007135212272565109, 0.001418278776876383, 0.001831065863439108, 0.003304255881575469], [0.0019479887016655303, 0.008523695874531197, 0.004640801930573603, 0.0052781589781484214, 0.2242152466367713, 0.001826417299824664, 0.001994157119639456, 0.016482610845557938, 0.007135212272565109, 0.001418278776876383, 0.001831065863439108, 0.003304255881575469], [0.0019479887016655303, 0.008523695874531197, 0.004640801930573603, 0.0052781589781484214, 0.2242152466367713, 0.001826417299824664, 0.006979549918738098, 0.016482610845557938, 0.007135212272565109, 0

In [11]:
# viterbi Algorithm

def viterbiAlgorithm(
    transition_probability,emission_probability,initial_state_probability,
    count_of_each_pos_tags,pos_tags,words,most_frequent_tag,testX):
  count_of_distinct_pos_tags = len(pos_tags)

  # declaring matrix to calculate probability at each k
  prob_ans_seq = []
  cursen = testX.split(" ")
  curlen = len(cursen)
  predicted_seq_of_pos_tags = []

  for i in range(0,count_of_distinct_pos_tags):
    t = []
    for j in range(0,curlen):
      t.append([0,"a","a"])
    prob_ans_seq.append(t)

  # end of matrix creation
  for j in pos_tags.keys():
    if cursen[0] in words.keys():
      prob_ans_seq[pos_tags[j]][0][0] = initial_state_probability[pos_tags[j]]*emission_probability[words[cursen[0]]][pos_tags[j]]
      prob_ans_seq[pos_tags[j]][0][1] = "start"
      prob_ans_seq[pos_tags[j]][0][2] = j
    else:
      prob_ans_seq[pos_tags[j]][0][0] = initial_state_probability[pos_tags[j]]
      prob_ans_seq[pos_tags[j]][0][1] = "start"
      prob_ans_seq[pos_tags[j]][0][2] = j

  for i in range(1,curlen):
    if cursen[i] in words.keys():
      for j in range(0,count_of_distinct_pos_tags):
        prev_prob = prob_ans_seq[j][i-1][0]
        prev_prev_postag = prob_ans_seq[j][i-1][1]
        prev_postag = prob_ans_seq[j][i-1][2]

        #print(cursen[i],i,prev_postag)
        for k in pos_tags.keys():
          max_prob = prob_ans_seq[pos_tags[k]][i][0]
          max_postag = prob_ans_seq[pos_tags[k]][i][1]

          if(prev_prev_postag == "start"):
            p1 = initial_state_probability[pos_tags[prev_postag]]
          else:
            p1 = transition_probability[pos_tags[prev_prev_postag]][pos_tags[prev_postag]]

          p2 = transition_probability[pos_tags[prev_postag]][pos_tags[k]]
          p3 = emission_probability[words[cursen[i]]][pos_tags[k]]

          cur_prob = prev_prob + math.log(p1 + 1) + math.log(p2 + 1) + math.log(p3 + 1)

          if max_prob <= cur_prob:
            max_prob = cur_prob
            prob_ans_seq[pos_tags[k]][i][0] = max_prob
            prob_ans_seq[pos_tags[k]][i][1] = prev_postag
            prob_ans_seq[pos_tags[k]][i][2] = k
    else:
      for j in range(0,count_of_distinct_pos_tags):
        prev_prob = prob_ans_seq[j][i-1][0]
        prev_prev_postag = prob_ans_seq[j][i-1][1]
        prev_postag = prob_ans_seq[j][i-1][2]

        for k in pos_tags.key():
          if (k == most_frequent_tag):
            max_prob = prob_ans_seq[pos_tags[k]][i][0]
            max_postag = prob_ans_seq[pos_tags[k]][i][1]

            if(prev_prev_postag == "start"):
              p1 = initial_state_probability[pos_tags[prev_postag]]
            else:
              p1 = transition_probability[pos_tags[prev_prev_postag]][pos_tags[prev_postag]]

            p2 = transition_probability[pos_tags[prev_postag]][pos_tags[k]]

            cur_prob = prev_prob + math.log(p1 + 1) + math.log(p2 + 1)

            if max_prob <= cur_prob:
              max_prob = cur_prob
              prob_ans_seq[pos_tags[k]][i][0] = max_prob
              prob_ans_seq[pos_tags[k]][i][1] = prev_postag
              prob_ans_seq[pos_tags[k]][i][2] = k

          else:
            prob_ans_seq[pos_tags[k]][i][0] = float(0)
            prob_ans_seq[pos_tags[k]][i][1] = prev_postag
            prob_ans_seq[pos_tags[k]][i][2] = k

    # print(prob_and_seq[pos_tags[k]][i][1])

  maxiprob = prob_ans_seq[0][curlen - 1][0]
  prevpostag = prob_ans_seq[0][curlen - 1][1]
  maxipostag = prob_ans_seq[0][curlen - 1][2]

  for i in range(0,count_of_distinct_pos_tags):
    if prob_ans_seq[i][curlen - 1][0] >= maxiprob:
      maxiprob = prob_ans_seq[i][curlen - 1][0]
      prevpostag = prob_ans_seq[i][curlen - 1][1]
      maxipostag = prob_ans_seq[i][curlen - 1][2]

  predicted_seq_of_pos_tags.append(maxipostag)

  for i in range(curlen - 2, -1, -1):
    predicted_seq_of_pos_tags.append(prevpostag)
    prevpostag = prob_ans_seq[pos_tags[prevpostag]][i][1]

  predicted_seq_of_pos_tags.reverse()

  # print(predicted_seq_of_pos_tags)

  return predicted_seq_of_pos_tags



In [13]:
# Running Viterbi algo to predict the POS tags(36) associated with each word of a sentence

predicted_Y = []
for i in X_test:
  predicted_Y.append(
      viterbiAlgorithm(
          transition_probability,emission_probability,
          initial_state_probability,count_of_each_pos_tags,
          pos_tags,words,most_frequent_tag,i))


In [14]:
# Calculating Accuracy

k = 0
accuracy = []
for i in Y_test:
  count = 0
  actualtags_Y = i.split(",")
  curlen = len(actualtags_Y)
  for j in range(0,curlen):
    if predicted_Y[k][j]==actualtags_Y[j]:
      count +=1
  k +=1
  accuracy.append(count/curlen)

print("Average accuracy before collapsing to 4 POS tags= ",sum(accuracy)*100/len(accuracy))

Average accuracy before collapsing to 4 POS tags=  89.31234465617233


In [15]:
# Applying POS tagging by collapsing 36 POS tags to 4 POS tags
# Mapping 36 Tags to 4 tags
# Creating transition matrix with 4 POS tags

map_36_to_4_pos = {'NNPS': "N",'VBG': "V",'VBD': "V",'X':"O",'PDT': "O","PRON":"O",
                   "NUM":"O","ADV":"A",'.':"O","PRT":"O","CONJ":"O","ADJ":"A",
                   "VERB":"V",'NOUN':"N",'DET':'O', 'PRP': "O", 'PRP$': "O",'ADP':"A",
                   'WP': "O", 'JJR': "A", 'JJ': "A", 'SYM': "O", 'NNP': "N", 'MD': "V",
                   'IN': "O", 'RP': "O", 'EX': "O", 'FW': "O", 'CD': "O", 'VBZ': "V",
                   'RBR': "A", 'NN': "N", 'TO': "O", 'LS': "O", 'VB': "V", 'DT': "O",
                   'WDT': "O", 'VBN': "V", 'WP$': "O", 'WRB': "O", 'NNS': "N", 'UH': "O",
                   'RBS': "A", 'JJS': "A", 'VBP': "V", 'CC': "O", 'RB': "A"}

count_of_distinct_pos_tags = 4
new_pos_tags = {"N": 0,"V": 1,"A": 2,"O":3}

#declaring transition matrix
transition_count_4 = []
for i in range(0,count_of_distinct_pos_tags):
  t = []
  for j in range(0,count_of_distinct_pos_tags):
    t.append(float(0))
  transition_count_4.append(t)

count_of_each_pos_tags_4 = [0]*count_of_distinct_pos_tags
pos_tags_as_initial_count_4 = [0]*count_of_distinct_pos_tags
count  = 0
for i in Y_train:
  line = list(i.split(","))
  pos_tags_as_initial_count_4[new_pos_tags[map_36_to_4_pos[line[0]]]]+=1
  count_of_each_pos_tags_4[new_pos_tags[map_36_to_4_pos[line[0]]]]+=1
  length  = int(len(line))
  for j in range(1,length):
    count_of_each_pos_tags_4[new_pos_tags[map_36_to_4_pos[line[j]]]]+=1
    transition_count_4[new_pos_tags[map_36_to_4_pos[line[j-1]]]][new_pos_tags[map_36_to_4_pos[line[j]]]] += 1

In [17]:
# Calculating initial state probability with 4 POS tags

count_of_distinct_pos_tags = 4
initial_state_probability_4 = [0]*count_of_distinct_pos_tags
for i in range(count_of_distinct_pos_tags):
  initial_state_probability_4[i] = (pos_tags_as_initial_count_4[i] + 1)/count_of_each_pos_tags_4[i]

In [18]:
# Calculating transition probability with 4 POS tags

transition_probability_4 = copy.deepcopy(transition_count_4)
count_of_distinct_pos_tags = 4
for i in range(0,count_of_distinct_pos_tags ):
  for j in range(0,count_of_distinct_pos_tags ):
    transition_probability_4[i][j] = (transition_probability_4[i][j] + 1)*100/(count_of_each_pos_tags_4[j] + 1)

print(transition_probability_4)


[[0.0009970686182623088, 0.0014182586620147782, 0.0009383944071693332, 0.0006363144666093984], [0.0009970686182623088, 0.0014182586620147782, 0.0009383944071693332, 0.0006363144666093984], [0.0009970686182623088, 0.0014182586620147782, 0.0009383944071693332, 0.0006363144666093984], [0.0009970686182623088, 0.0014182586620147782, 0.0009383944071693332, 0.0006363144666093984]]


In [19]:
#Calculating emission count matrix with 4 POS tags

count_of_distinct_words = len(words)
count_of_distinct_pos_tags = 4

# declaring emission count matrix with 4 POS tags
emission_count_4 = []
for i in range(0,count_of_distinct_words):
  t = []
  for j in range(0,count_of_distinct_pos_tags):
    t.append(float(0))
  emission_count_4.append(t)

for i in range(0,len(X_train)):
  lineX = X_train[i].split(" ")
  lineY = Y_train[i].split(",")
  length = len(lineX)

  for j in range(0,length):
    emission_count_4[words[lineX[j]]][new_pos_tags[map_36_to_4_pos[lineY[j-1]]]] += 1

In [20]:
# Calculating emission probability with 4 POS tags

emission_probability_4 = copy.deepcopy(emission_count_4)
count_of_distinct_pos_tags = 4

for i in range(0,count_of_distinct_pos_tags):
  for j in range(0,count_of_distinct_pos_tags):
    emission_probability_4[i][j] = ((emission_probability_4[i][j] + 1)*(100))/(count_of_each_pos_tags_4[j])

print(emission_probability_4[0])

[0.001994157119639456, 0.001418278776876383, 0.0009384032130926016, 0.0006363185155961669]


In [21]:
# Finding most frequent word

count_of_distinct_pos_tags = 4
for i in range(0,count_of_distinct_pos_tags):
  if(count_of_each_pos_tags_4[i]==max(count_of_each_pos_tags_4)):
    most_frequent_tag_index_4 = i
    break
print(most_frequent_tag_index_4)

for i in new_pos_tags:
  if(new_pos_tags[i]==most_frequent_tag_index_4):
    most_frequent_tag_4 = i
print(most_frequent_tag_4)

3
O


In [23]:
# Running Viterbi algo to predict the POS tags(4) associated with each word of a sentence

predicted_Y_4 = []
for i in X_test:
  predicted_Y_4.append(
      viterbiAlgorithm(
          transition_probability_4,emission_probability_4,
          initial_state_probability_4,count_of_each_pos_tags_4,
          new_pos_tags,words,most_frequent_tag_4,i))

In [24]:
#Calculating accuracy with 4 POS tags
#Calculating overall accuracy

k = 0
accuracy_4 = []
for i in Y_test:
  count = 0
  actualtags_Y_4 = i.split(",")
  curlen = len(actualtags_Y_4)
  for j in range(0,curlen):
    if predicted_Y_4[k][j]==map_36_to_4_pos[actualtags_Y_4[j]]:
      count +=1
  k +=1
  accuracy_4.append(count/curlen)
print(accuracy_4)

print(sum(accuracy_4)/len(accuracy_4))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [25]:
#Calculating class wise accuracy

correctpredict = {"N": 0,"V": 0,"A": 0,"O":0}
totalprediction = {"N": 0,"V": 0,"A": 0,"O":0}
k = 0
accuracy_4 = []
for i in Y_test:
  count = 0
  actualtags_Y_4 = i.split(",")
  curlen = len(actualtags_Y_4)
  for j in range(0,curlen):
    s = predicted_Y_4[k][j]
    t = map_36_to_4_pos[actualtags_Y_4[j]]
    if s==t:
      count += 1
      correctpredict[t] +=1
    totalprediction[t]+=1
  k +=1
  accuracy_4.append(count/curlen)

print("Average accuracy after collapsing to 4 POS tags = ",sum(accuracy_4)/len(accuracy_4))

print("\nClass wise prediction after collapsing to 4 POS tags")
for i in correctpredict:
  print(i," = ",(correctpredict[i]*100)/totalprediction[i])

Average accuracy after collapsing to 4 POS tags =  0.9121605449691613

Class wise prediction after collapsing to 4 POS tags
N  =  74.03916589344142
V  =  96.43784401287776
A  =  89.31160965623134
O  =  97.72492244053774
